In [1]:
import joblib
import numpy as np
from sentence_transformers import SentenceTransformer
import sys
import os
import textwrap

sys.path.append(os.path.abspath('..'))
project_root = os.path.dirname(os.path.dirname(os.path.abspath('.')))
if project_root not in sys.path:
    sys.path.append(project_root)

from train.analogy.config import MODEL_OUTPUT_DIR, MODEL_NAME
from src.logger import logger
from src import config as main_config

e:\agentic-reasoning-engine\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
try:
    # Load the artifact
    model_path = os.path.join(MODEL_OUTPUT_DIR, MODEL_NAME)
    artifact = joblib.load(model_path)
    logger.info(f"Loaded analogical reasoner artifact from {model_path}")

    # Extract components
    search_index = artifact['search_index']
    index_to_data_map = artifact['index_to_data_map']

    # Load the sentence transformer for inference
    embedding_model = SentenceTransformer(main_config.EMBEDDING_MODEL_NAME)
    logger.info(f"Sentence transformer '{main_config.EMBEDDING_MODEL_NAME}' is ready.")

except FileNotFoundError as e:
    logger.error(f"Error: {e}. Please run the training script first.")

2025-10-07 22:40:48,778 - AgenticReasoningPipeline - INFO - Loaded analogical reasoner artifact from e:\agentic-reasoning-engine\models\analogical_reasoner.pkl (1750854353.py:5)
2025-10-07 22:40:52,993 - AgenticReasoningPipeline - INFO - Sentence transformer 'all-MiniLM-L6-v2' is ready. (1750854353.py:13)


In [4]:
def find_analogies(query_text: str, k: int = 3):
    """Finds and displays the top-k most similar problems to a query text."""
    
    print("-" * 80)
    print("\n🔍 QUERY PROBLEM:")
    print(textwrap.fill(query_text, width=80))
    
    # 1. Create embedding for the query
    query_embedding = embedding_model.encode([query_text])
    
    # 2. Use the search index to find nearest neighbors
    distances, indices = search_index.kneighbors(query_embedding)
    
    print("\n⬇️ TOP ANALOGIES FOUND:")
    # 3. Retrieve and display the results
    for i, (idx, dist) in enumerate(zip(indices[0], distances[0])):
        retrieved_data = index_to_data_map[idx]
        similarity = 1 - dist # For cosine metric, similarity is 1 - distance
        
        print(f"\n--- Analogy #{i+1} (Similarity: {similarity:.4f}) ---")
        print("  [SIMILAR PROBLEM]:")
        print(textwrap.fill(retrieved_data['problem_statement'], width=78, initial_indent="    ", subsequent_indent="    "))
        print("\n  [SOLUTION]:")
        print(textwrap.fill(retrieved_data['solution'], width=78, initial_indent="    ", subsequent_indent="    "))
    print("\n" + "-" * 80)

In [5]:
def find_analogies_detailed(query_text: str, k: int = 3):
    """Finds similar problems with additional metadata and formatting."""
    
    print("=" * 100)
    print("🧠 ANALOGICAL REASONING ENGINE")
    print("=" * 100)
    print("\n📋 QUERY PROBLEM:")
    print(textwrap.fill(query_text, width=90))
    
    # Create embedding and search
    query_embedding = embedding_model.encode([query_text])
    distances, indices = search_index.kneighbors(query_embedding, n_neighbors=k)
    
    print(f"\n🎯 FOUND {len(indices[0])} ANALOGIES:")
    print("-" * 100)
    
    for i, (idx, dist) in enumerate(zip(indices[0], distances[0])):
        retrieved_data = index_to_data_map[idx]
        similarity = (1 - dist) * 100  # Convert to percentage
        
        print(f"\n🔹 ANALOGY #{i+1} | Similarity: {similarity:.1f}%")
        print(f"   Problem Type: {retrieved_data.get('topic', 'N/A')}")
        print(f"\n   📝 SIMILAR PROBLEM:")
        wrapped_problem = textwrap.fill(
            retrieved_data['problem_statement'], 
            width=85, 
            initial_indent="     ", 
            subsequent_indent="     "
        )
        print(wrapped_problem)
        
        print(f"\n   💡 SOLUTION:")
        wrapped_solution = textwrap.fill(
            retrieved_data['solution'], 
            width=85, 
            initial_indent="     ", 
            subsequent_indent="     "
        )
        print(wrapped_solution)
        print("   " + "-" * 50)
    
    print("\n" + "=" * 100)

In [6]:
# Test Case 1: A problem very similar to one in our dataset
query1 = "A traveler needs to visit Paris, Berlin, and Rome, starting and ending in Paris. What's the shortest trip if P->B is 100km, B->R is 120km, and P->R is 150km?"
find_analogies_detailed(query1, k=2)

🧠 ANALOGICAL REASONING ENGINE

📋 QUERY PROBLEM:
A traveler needs to visit Paris, Berlin, and Rome, starting and ending in Paris. What's
the shortest trip if P->B is 100km, B->R is 120km, and P->R is 150km?

🎯 FOUND 2 ANALOGIES:
----------------------------------------------------------------------------------------------------

🔹 ANALOGY #1 | Similarity: 69.8%
   Problem Type: N/A

   📝 SIMILAR PROBLEM:
     A traveler needs to visit four cities – A, B, C, and D. The distances between
     them are: A to B is 300km, A to C is 200km, A to D is 400km, B to C is 150km, B
     to D is 250km, C to D is 100km. The traveler can start at any city. What is the
     shortest path he can take to visit all cities, without visiting the same city
     more than once?

   💡 SOLUTION:
     The shortest path is A -> C -> D -> B -> A which is 200km + 100km + 250km +
     300km = 850km. The process is to list all permutations of visits and calculate
     the sum of distances for each. Then, choose the pe

In [7]:
# Test Case 2: A riddle with different wording
query2 = "If a photographer takes a picture of her spouse, develops the film in water, and hangs the photo up to dry, why can they go to dinner together that night?"
find_analogies_detailed(query2)

🧠 ANALOGICAL REASONING ENGINE

📋 QUERY PROBLEM:
If a photographer takes a picture of her spouse, develops the film in water, and hangs the
photo up to dry, why can they go to dinner together that night?

🎯 FOUND 3 ANALOGIES:
----------------------------------------------------------------------------------------------------

🔹 ANALOGY #1 | Similarity: 60.8%
   Problem Type: N/A

   📝 SIMILAR PROBLEM:
     A woman shoots her husband. Then she holds him underwater for over 5 minutes.
     Finally, she hangs him. But 10 minutes later they both go out and enjoy a
     wonderful dinner together. How can this be?

   💡 SOLUTION:
     The woman shot her husband with a camera, took a picture. She then developed the
     photo in her dark room where the photograph paper was submerged in the solution.
     'Hanging' referred to putting the photo up to dry.
   --------------------------------------------------

🔹 ANALOGY #2 | Similarity: 32.6%
   Problem Type: N/A

   📝 SIMILAR PROBLEM:
     Mark

In [8]:
# Test Case 3: A completely new spatial reasoning problem
query3 = "You are standing in a hallway with a door on the left and a door on the right. The left door leads to a library and the right door leads to a kitchen. How do you get to the garden if the kitchen has a back door?"
find_analogies_detailed(query3)

🧠 ANALOGICAL REASONING ENGINE

📋 QUERY PROBLEM:
You are standing in a hallway with a door on the left and a door on the right. The left
door leads to a library and the right door leads to a kitchen. How do you get to the
garden if the kitchen has a back door?

🎯 FOUND 3 ANALOGIES:
----------------------------------------------------------------------------------------------------

🔹 ANALOGY #1 | Similarity: 60.9%
   Problem Type: N/A

   📝 SIMILAR PROBLEM:
     You are in a room with 4 doors, each door is painted with a different color;
     red, blue, green, and yellow. Each door leads to a room of the same color. Once
     you enter a colored room, there is another door that leads back to the original
     room which is not necessarily the same color as the one you entered. The red
     room leads back through the green door, the green room leads back through the
     red door, the blue room leads back through the yellow door. Which door leads
     back to the original room when you 

In [9]:
def batch_test_analogies(test_queries):
    """Test multiple queries and provide a summary."""
    print("🧪 BATCH TESTING ANALOGICAL REASONER")
    print("=" * 100)
    
    results = []
    for i, query in enumerate(test_queries, 1):
        print(f"\n{'='*50} TEST CASE {i} {'='*50}")
        
        # Get the top result for each query
        query_embedding = embedding_model.encode([query])
        distances, indices = search_index.kneighbors(query_embedding, n_neighbors=1)
        
        top_similarity = (1 - distances[0][0]) * 100
        top_problem = index_to_data_map[indices[0][0]]['problem_statement'][:100] + "..."
        
        results.append({
            'query': query[:80] + "..." if len(query) > 80 else query,
            'top_similarity': top_similarity,
            'top_match_preview': top_problem
        })
        
        print(f"Query: {query}")
        print(f"Top Match Similarity: {top_similarity:.1f}%")
        print(f"Top Match Preview: {top_problem}")
    
    # Summary
    print(f"\n{'='*100}")
    print("📊 BATCH TEST SUMMARY")
    print(f"{'='*100}")
    avg_similarity = np.mean([r['top_similarity'] for r in results])
    max_similarity = np.max([r['top_similarity'] for r in results])
    min_similarity = np.min([r['top_similarity'] for r in results])
    
    print(f"Average Top Similarity: {avg_similarity:.1f}%")
    print(f"Highest Similarity: {max_similarity:.1f}%")
    print(f"Lowest Similarity: {min_similarity:.1f}%")
    print(f"Total Test Cases: {len(results)}")
    
    return results

# Run batch test
test_queries = [
    "How to measure 4 liters using 3-liter and 5-liter jugs?",
    "A man has to get a fox, chicken, and corn across a river in a small boat.",
    "Find the shortest path connecting all major cities in a region.",
    "If it takes 5 machines 5 minutes to make 5 widgets, how long for 100 machines?",
    "You have 8 balls, one is heavier. Find it with a balance scale in 2 weighs."
]

batch_results = batch_test_analogies(test_queries)

🧪 BATCH TESTING ANALOGICAL REASONER

================================================== TEST CASE 1 ==================================================
Query: How to measure 4 liters using 3-liter and 5-liter jugs?
Top Match Similarity: 66.1%
Top Match Preview: You have two water jugs: a 5-liter jug and a 7-liter jug, both empty initially. There's a tap that c...

================================================== TEST CASE 2 ==================================================
Query: A man has to get a fox, chicken, and corn across a river in a small boat.
Top Match Similarity: 45.9%
Top Match Preview: A man stands on one side of a river, his dog on the other. The man calls his dog, who immediately cr...

================================================== TEST CASE 3 ==================================================
Query: Find the shortest path connecting all major cities in a region.
Top Match Similarity: 69.2%
Top Match Preview: A traveler needs to visit four cities – A, B, C, and D.

In [11]:
def assess_reasoner_quality():
    """Assess the overall quality of the analogical reasoner."""
    print("📈 ANALOGICAL REASONER QUALITY ASSESSMENT")
    print("=" * 100)
    
    # Test with known similar pairs from training data
    sample_problems = index_to_data_map[:10]  # Fixed: index_to_data_map is a list
    
    print("Testing retrieval accuracy on training samples...")
    accuracies = []
    
    for i, sample in enumerate(sample_problems[:5]):
        query = sample['problem_statement']
        query_embedding = embedding_model.encode([query])
        distances, indices = search_index.kneighbors(query_embedding, n_neighbors=5)
        
        # Check if the exact same problem is in top results (should be very high similarity)
        exact_match_idx = None
        for j, idx in enumerate(indices[0]):
            if index_to_data_map[idx]['problem_statement'] == query:
                exact_match_idx = j
                break
        
        if exact_match_idx is not None:
            accuracy = 1.0  # Perfect self-match
        else:
            # Calculate average similarity to top 5
            accuracy = np.mean([1 - dist for dist in distances[0]])
        
        accuracies.append(accuracy)
        
        print(f"Sample {i+1}: Top similarity = {(1-distances[0][0])*100:.1f}% | "
              f"Exact match rank = {exact_match_idx+1 if exact_match_idx is not None else 'N/A'}")
    
    avg_accuracy = np.mean(accuracies)
    print(f"\nOverall Quality Score: {avg_accuracy:.3f}")
    
    if avg_accuracy > 0.8:
        print("✅ Reasoner Quality: EXCELLENT")
    elif avg_accuracy > 0.6:
        print("✅ Reasoner Quality: GOOD")
    elif avg_accuracy > 0.4:
        print("⚠️  Reasoner Quality: FAIR")
    else:
        print("❌ Reasoner Quality: POOR")
    
    return avg_accuracy

# Run quality assessment
quality_score = assess_reasoner_quality()

📈 ANALOGICAL REASONER QUALITY ASSESSMENT
Testing retrieval accuracy on training samples...
Sample 1: Top similarity = 100.0% | Exact match rank = 1
Sample 2: Top similarity = 100.0% | Exact match rank = 1
Sample 3: Top similarity = 100.0% | Exact match rank = 1
Sample 4: Top similarity = 100.0% | Exact match rank = 1
Sample 5: Top similarity = 100.0% | Exact match rank = 1

Overall Quality Score: 1.000
✅ Reasoner Quality: EXCELLENT
